In [ ]:
import torch
import torch.nn as nn

##Creating model

In [ ]:
class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__() #import constructor from parent class
    self.network = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)
    return out

In [ ]:
#Create dataset
features = torch.rand(10, 5)

In [ ]:
#create model
model = Model(features.shape[1])

In [ ]:
#call model for forward pass
model(features)

tensor([[0.3589],
        [0.3713],
        [0.3799],
        [0.3611],
        [0.4039],
        [0.3876],
        [0.3819],
        [0.3677],
        [0.3896],
        [0.3907]], grad_fn=<SigmoidBackward0>)

#Let's Improve previous Code

In [ ]:
import torch.nn as nn
class MySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    output = self.linear(features)
    output = self.sigmoid(output)
    return output


In [ ]:
#important parameters
learning_rate = 0.1
epochs = 5

In [ ]:
#loss function
loss_function = nn.BCELoss()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

#load data
url = "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv"
df = pd.read_csv(url)

#drop columns
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

#split
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

#train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#StandardSclr
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#LabelEncoding
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Convert Numpy array to pytorch
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train_encoded)
y_test_tensor = torch.from_numpy(y_test_encoded)

In [ ]:
X_train_tensor = X_train_tensor.float()
X_test_tensor = X_test_tensor.float()
y_train_tensor = y_train_tensor.float()
y_test_tensor = y_test_tensor.float()

In [ ]:
#training pipeline
model = MySimpleNN(X_train_tensor.shape[1])


In [ ]:
#optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  y_pred = model(X_train_tensor)
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  acc = (y_pred == y_test_tensor).float().mean()

In [ ]:
acc

tensor(0.5269)